## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-25-20-48-46 +0000,nypost,Trump signs TikTok deal that gives American in...,https://nypost.com/2025/09/25/business/trump-s...
1,2025-09-25-20-48-36 +0000,nyt,"Lisa Cook Asks Supreme Court to Remain at Fed,...",https://www.nytimes.com/2025/09/25/us/politics...
2,2025-09-25-20-45-43 +0000,wapo,Trump administration calls for radical reform ...,https://www.washingtonpost.com/national-securi...
3,2025-09-25-20-40-00 +0000,wsj,Stocks Extend Slide for a Third-Straight Day A...,https://www.wsj.com/finance/stocks/stocks-exte...
4,2025-09-25-20-39-27 +0000,bbc,Jaguar Land Rover restarts some IT systems aft...,https://www.bbc.com/news/articles/c0q75q4l87no...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2354/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,50
89,ice,19
12,president,18
102,shooting,15
88,dallas,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
152,2025-09-25-14-00-00 +0000,wsj,The Trump administration will offer AI models ...,https://www.wsj.com/tech/ai/trump-elon-musk-xa...,123
37,2025-09-25-19-27-11 +0000,nypost,White House reposts Tylenol’s own old tweet wa...,https://nypost.com/2025/09/25/us-news/white-ho...,108
81,2025-09-25-17-49-10 +0000,nypost,Trump calls out Turkish president for buying R...,https://nypost.com/2025/09/25/us-news/trump-ca...,103
112,2025-09-25-16-25-34 +0000,nypost,Disney gears up for court battle with Trump af...,https://nypost.com/2025/09/25/business/disney-...,102
330,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com,101


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
152,123,2025-09-25-14-00-00 +0000,wsj,The Trump administration will offer AI models ...,https://www.wsj.com/tech/ai/trump-elon-musk-xa...
48,86,2025-09-25-19-05-21 +0000,nypost,Anti-ICE gunman Joshua Jahn had ‘hatred’ for g...,https://nypost.com/2025/09/25/us-news/anti-ice...
164,62,2025-09-25-13-09-54 +0000,bbc,Former French president Sarkozy given five-yea...,https://www.bbc.com/news/articles/cp98kepmj9lo...
114,56,2025-09-25-16-17-15 +0000,nypost,Over 100 ex-ABC journalists urge Disney’s Bob ...,https://nypost.com/2025/09/25/business/over-10...
301,53,2025-09-24-23-28-04 +0000,nyt,Dallas ICE Facility Shooting Leaves One Detain...,https://www.nytimes.com/2025/09/24/us/dallas-i...
95,41,2025-09-25-17-10-56 +0000,nypost,Chinese drone experts have been working with R...,https://nypost.com/2025/09/25/world-news/chine...
175,38,2025-09-25-12-27-47 +0000,nyt,"Starbucks to Cut Jobs, Close Stores and Take $...",https://www.nytimes.com/2025/09/25/business/st...
6,37,2025-09-25-20-31-36 +0000,nyt,Trump Assures Arab Leaders He Won’t Allow Isra...,https://www.nytimes.com/2025/09/25/world/middl...
22,35,2025-09-25-20-05-17 +0000,nypost,Secretary of War Pete Hegseth orders nearly al...,https://nypost.com/2025/09/25/us-news/secretar...
27,34,2025-09-25-19-49-13 +0000,bbc,Amazon agrees to pay $2.5bn over claims it tri...,https://www.bbc.com/news/articles/cg7dy9018jxo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
